# Twitter sentiment analysis

In [1]:
# Install Kaggle Library
!pip install kaggle

### Upload Kaggle.json

In [5]:
# Configuring the path of kaggle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

### Importing Twitter Sentiment dataset

In [6]:
# API to fetch the dataset from kaggle
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
  0% 0.00/80.9M [00:00<?, ?B/s]
100% 80.9M/80.9M [00:00<00:00, 1.53GB/s]


In [7]:
# extracting the compressed dataset
from zipfile import ZipFile
dataset = '/content/sentiment140.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


## Importing the Dependencies

In [8]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [9]:
import nltk
nltk.download('stopwords') # words that doesn't have any meaning to my textual data

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [10]:
# printing the stopwords in English
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

## Data Processing

In [11]:
# loading the data from csv file to pandas dataframe
# use standard encoding
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1')


In [12]:
# checking the number of rows and columns
twitter_data.shape

(1599999, 6)

### Data Columns:
- target: the polarity of the tweet (0 = negative, 2 = neutral, 4 = positive)
- ids: The id of the tweet ( 2087)
- date: the date of the tweet (Sat May 16 23:58:44 UTC 2009)
- flag: The query (lyx). If there is no query, then this - value is NO_QUERY.
- user: the user that tweeted (robotickilldozr)
- text: the text of the tweet (Lyx is cool)

In [13]:
# printing the first 5 rows of the dataframe
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [14]:
# naming the coloumns and reading the dataset again
column_names = ['target','ids','date','flag','user','text']
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1',names=column_names)

In [15]:
# checking the number of rows and columns
twitter_data.shape

(1600000, 6)

In [16]:
# printing the first 5 rows of the dataframe
twitter_data.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [17]:
# counting the number of missing values in the dataset
twitter_data.isnull().sum()

,0
target,0
ids,0
date,0
flag,0
user,0
text,0


In [18]:
# checking the distribution of target coloumn
twitter_data.target.value_counts()

,count
target,
0,800000
4,800000


### Convert target 4 to 1

In [20]:
twitter_data.replace({'target': {4:1}},inplace=True)

In [21]:
# checking the distribution of target coloumn
twitter_data.target.value_counts()

,count
target,
0,800000
1,800000


**Tweets:**
- 0 : Negative Tweet
- 1 : Positive Tweet

**Stemming:**
- process that we do in order to reduce a word to its root word.
- example: actor, actress, acting = act


In [22]:
port_stem = PorterStemmer()

In [23]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content


In [24]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)

In [25]:
# after 50 minutes from the previous steps
twitter_data.head()

,target,ids,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [26]:
print(twitter_data['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [27]:
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [28]:
# Seperating the data and label
X = twitter_data['stemmed_content'].values
Y = twitter_data['target'].values

In [29]:
print(X)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [30]:
print(Y)

[0 0 0 ... 1 1 1]


### Splitting data to training data and test data

In [31]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)

In [32]:
print(X.shape, X_train.shape, X_test.shape)

(1600000,) (1280000,) (320000,)


In [33]:
print(X_train)

['watch saw iv drink lil wine' 'hatermagazin'
 'even though favourit drink think vodka coke wipe mind time think im gonna find new drink'
 ... 'eager monday afternoon'
 'hope everyon mother great day wait hear guy store tomorrow'
 'love wake folger bad voic deeper']


In [34]:
print(X_test)

['mmangen fine much time chat twitter hubbi back summer amp tend domin free time'
 'ah may show w ruth kim amp geoffrey sanhueza'
 'ishatara mayb bay area thang dammit' ...
 'destini nevertheless hooray member wonder safe trip' 'feel well'
 'supersandro thank']


### Converting the textual data in to numerical values

In [35]:
vectorizer = TfidfVectorizer()
x_train = vectorizer.fit_transform(X_train)
x_test = vectorizer.transform(X_test)

In [36]:
print(x_train)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 9453092 stored elements and shape (1280000, 461488)>
  Coords	Values
  (0, 436713)	0.27259876264838384
  (0, 354543)	0.3588091611460021
  (0, 185193)	0.5277679060576009
  (0, 109306)	0.3753708587402299
  (0, 235045)	0.41996827700291095
  (0, 443066)	0.4484755317023172
  (1, 160636)	1.0
  (2, 109306)	0.4591176413728317
  (2, 124484)	0.1892155960801415
  (2, 407301)	0.18709338684973031
  (2, 129411)	0.29074192727957143
  (2, 406399)	0.32105459490875526
  (2, 433560)	0.3296595898028565
  (2, 77929)	0.31284080750346344
  (2, 443430)	0.3348599670252845
  (2, 266729)	0.24123230668976975
  (2, 409143)	0.15169282335109835
  (2, 178061)	0.1619010109445149
  (2, 150715)	0.18803850583207948
  (2, 132311)	0.2028971570399794
  (2, 288470)	0.16786949597862733
  (3, 406399)	0.29029991238662284
  (3, 158711)	0.4456939372299574
  (3, 151770)	0.278559647704793
  (3, 56476)	0.5200465453608686
  :	:
  (1279996, 318303)	0.21254698865277744
  (12

In [37]:
print(x_test)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2289192 stored elements and shape (320000, 461488)>
  Coords	Values
  (0, 15110)	0.1719352837797837
  (0, 31168)	0.1624772418052177
  (0, 67828)	0.26800375270827315
  (0, 106069)	0.36555450010904555
  (0, 132364)	0.255254889555786
  (0, 138164)	0.23688292264071406
  (0, 171378)	0.2805816206356074
  (0, 271016)	0.45356623916588285
  (0, 279082)	0.17825180109103442
  (0, 388348)	0.2198507607206174
  (0, 398906)	0.34910438732642673
  (0, 409143)	0.3143047059807971
  (0, 420984)	0.17915624523539805
  (1, 6463)	0.30733520460524466
  (1, 15110)	0.211037449588008
  (1, 145393)	0.575262969264869
  (1, 217562)	0.40288153995289894
  (1, 256777)	0.28751585696559306
  (1, 348135)	0.4739279595416274
  (1, 366203)	0.24595562404108307
  (2, 22532)	0.3532582957477176
  (2, 34401)	0.37916255084357414
  (2, 89448)	0.36340369428387626
  (2, 183312)	0.5892069252021465
  (2, 256834)	0.2564939661498776
  :	:
  (319994, 443794)	0.2782185641032538


### Training the Machine Learning Model

#### Logistic Regression

In [38]:
model = LogisticRegression(max_iter=1000)

In [40]:
model.fit(x_train, Y_train)

LogisticRegression(max_iter=1000)

## Model Evaluation

### Accuracy Score

In [70]:
# accuracy score in the training data
X_train_prediction = model.predict(x_train)

In [71]:
training_data_accuracy = accuracy_score(Y_train, X_train_prediction)

In [72]:
print("Accuracy score in the training data", training_data_accuracy)

Accuracy score in the training data 0.79871953125


In [73]:
# accuracy score in the test data
X_test_prediction = model.predict(x_test)

In [74]:
test_data_accuracy = accuracy_score(Y_test, X_test_prediction)

In [75]:
print("Accuracy score in the test data", test_data_accuracy)

Accuracy score in the test data 0.77668125


### It can be overfitting if the accuracy for test data is way less for train data ... data not proberly trained

In [77]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(Y_test, X_test_prediction))




              precision    recall  f1-score   support

           0       0.79      0.76      0.77    160000
           1       0.77      0.80      0.78    160000

    accuracy                           0.78    320000
   macro avg       0.78      0.78      0.78    320000
weighted avg       0.78      0.78      0.78    320000



In [78]:
cm = confusion_matrix(Y_test, X_test_prediction)
print(cm)

[[121246  38754]
 [ 32708 127292]]


## Saving the model

In [48]:
import pickle

In [49]:
file_name = 'trained_model.sav'
pickle.dump(model,open(file_name,'wb'))

### Using the Saved model for future predicition

In [55]:
# Loading the saved model
loaded_model = pickle.load(open('/content/trained_model.sav','rb'))

In [62]:
X_new = x_test[10] # x_test not X_test
print(X_test[10]) # to show the text
print(X_new)

caitlyndewar awesom got tan burn suck x
<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 6 stored elements and shape (1, 461488)>
  Coords	Values
  (0, 28957)	0.2771269162342073
  (0, 56409)	0.37580155550460353
  (0, 58606)	0.7083311938978187
  (0, 151770)	0.21723573981478025
  (0, 387426)	0.2864793774235901
  (0, 394913)	0.3885601402463296


In [63]:
prediction = loaded_model.predict(X_new)
print(prediction)

if (prediction[0] == 0):
  print('Negative Tweet')
else:
  print('Positive Tweet')

[0]
Negative Tweet


In [64]:
X_new = x_test[3] # x_test not X_test
print(X_test[3]) # to show the text
print(X_new)
prediction = loaded_model.predict(X_new)
print(prediction)

if (prediction[0] == 0):
  print('Negative Tweet')
else:
  print('Positive Tweet')

game end lost stinkyy
<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 3 stored elements and shape (1, 461488)>
  Coords	Values
  (0, 120049)	0.5801692086479822
  (0, 142563)	0.5742504216907061
  (0, 241674)	0.57761591263124
[0]
Negative Tweet


In [65]:
X_new = x_test[15] # x_test not X_test
print(X_test[15]) # to show the text
print(X_new)
prediction = loaded_model.predict(X_new)
print(prediction)

if (prediction[0] == 0):
  print('Negative Tweet')
else:
  print('Positive Tweet')

jaym smack ass anoth one fun
<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 6 stored elements and shape (1, 461488)>
  Coords	Values
  (0, 19780)	0.31622754607095854
  (0, 25990)	0.3808396243013339
  (0, 140518)	0.2792791113558299
  (0, 190595)	0.5731863923059995
  (0, 301929)	0.24164055723343197
  (0, 372455)	0.5385459111379511
[1]
Positive Tweet


In [66]:
X_new = x_test[40] # x_test not X_test
print(X_test[40]) # to show the text
print(X_new)
prediction = loaded_model.predict(X_new)
print(prediction)

if (prediction[0] == 0):
  print('Negative Tweet')
else:
  print('Positive Tweet')

ever hate stare phone whole life waitin ring prove alon
<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10 stored elements and shape (1, 461488)>
  Coords	Values
  (0, 12316)	0.2942599049463726
  (0, 124565)	0.2534136015312001
  (0, 160604)	0.2297799658315937
  (0, 234423)	0.24317756361108744
  (0, 315197)	0.2510973603080825
  (0, 325091)	0.40285080028193393
  (0, 341317)	0.34447765948775466
  (0, 381743)	0.38789008625609955
  (0, 435479)	0.4002159167797624
  (0, 441254)	0.2874430137805957
[0]
Negative Tweet


In [67]:
X_new = x_test[200] # x_test not X_test
print(X_test[200]) # to show the text
print(X_new)
prediction = loaded_model.predict(X_new)
print(prediction)

if (prediction[0] == 0):
  print('Negative Tweet')
else:
  print('Positive Tweet')

love sucki day get unexpect text put smile face
<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 9 stored elements and shape (1, 461488)>
  Coords	Values
  (0, 93795)	0.18766071909970514
  (0, 127090)	0.32853785623524473
  (0, 145988)	0.18586475941913905
  (0, 242268)	0.20103456365786704
  (0, 326966)	0.3047343739770557
  (0, 372988)	0.3474823166340872
  (0, 387466)	0.46923185113800253
  (0, 400002)	0.33425960056335163
  (0, 425173)	0.4861797169777241
[1]
Positive Tweet
